In [0]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "d31f62da-ae4a-4fb8-8f86-16d09a315f2f",
"fs.azure.account.oauth2.client.secret": 'os-8Q~agLf4EJAFjYye5pWN7SUKGUC7rg719pdsk',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/ed570910-d325-4362-a77a-1441a44f0cf3/oauth2/token"}


dbutils.fs.mount(
source = "abfss://paris-olympics-data@parisolympicsdatatab.dfs.core.windows.net", 
mount_point = "/mnt/parisolymics",
extra_configs = configs)

Out[1]: True

In [0]:
%fs
ls "/mnt/parisolymics"

path,name,size,modificationTime
dbfs:/mnt/parisolymics/raw-data/,raw-data/,0,1749052915000
dbfs:/mnt/parisolymics/transformed-data/,transformed-data/,0,1749052936000


In [0]:
from pyspark.sql.functions import col, regexp_replace

athletes_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("multiLine", "true") \
    .option("quote", '"') \
    .option("escape", '"') \
    .option("delimiter", ",") \
    .load("/mnt/parisolymics/raw-data/athletes.csv")

athletes_trimmed = athletes_raw.select(
    col("name"),
    col("country"),
    col("disciplines"),
    col("events")
)

athletes = athletes_trimmed.withColumn("disciplines", regexp_replace("disciplines", r"[\[\]'\"']", "")) \
                     .withColumn("events", regexp_replace("events", r"[\[\]'\"']", ""))


athletes.show(truncate=False)

+------------------------+--------+-----------------+-----------------------------------------------------+
|name                    |country |disciplines      |events                                               |
+------------------------+--------+-----------------+-----------------------------------------------------+
|ALEKSANYAN Artur        |Armenia |Wrestling        |Mens Greco-Roman 97kg                                |
|AMOYAN Malkhas          |Armenia |Wrestling        |Mens Greco-Roman 77kg                                |
|GALSTYAN Slavik         |Armenia |Wrestling        |Mens Greco-Roman 67kg                                |
|HARUTYUNYAN Arsen       |Armenia |Wrestling        |Mens Freestyle 57kg                                  |
|TEVANYAN Vazgen         |Armenia |Wrestling        |Mens Freestyle 65kg                                  |
|ARENAS Lorena           |Colombia|Athletics        |Womens 20km Race Walk, Marathon Race Walk Relay Mixed|
|McKENZIE Ashley         |Ja

In [0]:
display(athletes)

name,country,disciplines,events
ALEKSANYAN Artur,Armenia,Wrestling,Mens Greco-Roman 97kg
AMOYAN Malkhas,Armenia,Wrestling,Mens Greco-Roman 77kg
GALSTYAN Slavik,Armenia,Wrestling,Mens Greco-Roman 67kg
HARUTYUNYAN Arsen,Armenia,Wrestling,Mens Freestyle 57kg
TEVANYAN Vazgen,Armenia,Wrestling,Mens Freestyle 65kg
ARENAS Lorena,Colombia,Athletics,"Womens 20km Race Walk, Marathon Race Walk Relay Mixed"
McKENZIE Ashley,Jamaica,Judo,Men -60 kg
BASS BITTAYE Gina Mariam,Gambia,Athletics,"Womens 100m, Womens 200m"
CAMARA Ebrahima,Gambia,Athletics,Mens 100m
RUEDA SANTOS Lizeth,Mexico,Triathlon,Womens Individual


In [0]:
coaches_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load("/mnt/parisolymics/raw-data/coaches.csv")

coaches = coaches_raw.select(
    col("name"),
    col("gender"),
    col("country"),
    col("disciplines"),
    col("events")
)

coaches.printSchema()
coaches.show()

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- country: string (nullable = true)
 |-- disciplines: string (nullable = true)
 |-- events: string (nullable = true)

+--------------------+------+---------+-------------------+------+
|                name|gender|  country|        disciplines|events|
+--------------------+------+---------+-------------------+------+
|      PEDRERO Ofelia|Female|   Mexico|  Artistic Swimming|  Team|
|    RADHI SHENAISHIL|  Male|     Iraq|           Football|   Men|
| AFLAKIKHAMSEH Majid|  Male|  IR Iran|          Taekwondo|  null|
|     YOUSEFY Mehrdad|  Male|  IR Iran|          Taekwondo|  null|
|        MADDAH Minoo|Female|  IR Iran|          Taekwondo|  null|
|      LOFTUS Adriana|Female|   Mexico|  Artistic Swimming|  Team|
|    FERRARA Fernando|  Male|Argentina|             Hockey|  null|
|     GULLA Alejandra|Female|Argentina|             Hockey| Women|
|    CAPURRO Santiago|  Male|Argentina|             Hockey|  nu

In [0]:
display(coaches)

name,gender,country,disciplines,events
PEDRERO Ofelia,Female,Mexico,Artistic Swimming,Team
RADHI SHENAISHIL,Male,Iraq,Football,Men
AFLAKIKHAMSEH Majid,Male,IR Iran,Taekwondo,null
YOUSEFY Mehrdad,Male,IR Iran,Taekwondo,null
MADDAH Minoo,Female,IR Iran,Taekwondo,null
LOFTUS Adriana,Female,Mexico,Artistic Swimming,Team
FERRARA Fernando,Male,Argentina,Hockey,null
GULLA Alejandra,Female,Argentina,Hockey,Women
CAPURRO Santiago,Male,Argentina,Hockey,null
RONCONI Mariano,Male,Argentina,Hockey,null


In [0]:
medallists_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load("/mnt/parisolymics/raw-data/medallist.csv")

medallists = medallists_raw.select(
    col("medal_type"),
    col("name"),
    col("gender"),
    col("country"),
    col("event"),
    col("event_type")
)

medallists.printSchema()
medallists.show()

root
 |-- medal_type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- country: string (nullable = true)
 |-- event: string (nullable = true)
 |-- event_type: string (nullable = true)

+------------+--------------------+------+----------------+--------------------+----------+
|  medal_type|                name|gender|         country|               event|event_type|
+------------+--------------------+------+----------------+--------------------+----------+
|  Gold Medal|     EVENEPOEL Remco|  Male|         Belgium|Men's Individual ...|       ATH|
|Silver Medal|       GANNA Filippo|  Male|           Italy|Men's Individual ...|       ATH|
|Bronze Medal|       van AERT Wout|  Male|         Belgium|Men's Individual ...|       ATH|
|  Gold Medal|         BROWN Grace|Female|       Australia|Women's Individua...|       ATH|
|Silver Medal|      HENDERSON Anna|Female|   Great Britain|Women's Individua...|       ATH|
|Bronze Medal|        DY

In [0]:
display(medallists)

medal_type,name,gender,country,event,event_type
Gold Medal,EVENEPOEL Remco,Male,Belgium,Men's Individual Time Trial,ATH
Silver Medal,GANNA Filippo,Male,Italy,Men's Individual Time Trial,ATH
Bronze Medal,van AERT Wout,Male,Belgium,Men's Individual Time Trial,ATH
Gold Medal,BROWN Grace,Female,Australia,Women's Individual Time Trial,ATH
Silver Medal,HENDERSON Anna,Female,Great Britain,Women's Individual Time Trial,ATH
Bronze Medal,DYGERT Chloe,Female,United States,Women's Individual Time Trial,ATH
Gold Medal,OH Sanguk,Male,Korea,Men's Sabre Individual,HATH
Silver Medal,FERJANI Fares,Male,Tunisia,Men's Sabre Individual,HATH
Bronze Medal,SAMELE Luigi,Male,Italy,Men's Sabre Individual,HATH
Gold Medal,KONG Man Wai Vivian,Female,"Hong Kong, China",Women's Épée Individual,HATH


In [0]:
medalstotal_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load("/mnt/parisolymics/raw-data/medalstotal.csv")

medalstotal = medalstotal_raw.drop("country_code","country_long")

medalstotal.printSchema()
medalstotal.show()

root
 |-- country: string (nullable = true)
 |-- Gold Medal: integer (nullable = true)
 |-- Silver Medal: integer (nullable = true)
 |-- Bronze Medal: integer (nullable = true)
 |-- Total: integer (nullable = true)

+-------------+----------+------------+------------+-----+
|      country|Gold Medal|Silver Medal|Bronze Medal|Total|
+-------------+----------+------------+------------+-----+
|United States|        40|          44|          42|  126|
|        China|        40|          27|          24|   91|
|        Japan|        20|          12|          13|   45|
|    Australia|        18|          19|          16|   53|
|       France|        16|          26|          22|   64|
|  Netherlands|        15|           7|          12|   34|
|Great Britain|        14|          22|          29|   65|
|        Korea|        13|           9|          10|   32|
|        Italy|        12|          13|          15|   40|
|      Germany|        12|          13|           8|   33|
|  New Zealand|  

In [0]:
display(medalstotal)

country,Gold Medal,Silver Medal,Bronze Medal,Total
United States,40,44,42,126
China,40,27,24,91
Japan,20,12,13,45
Australia,18,19,16,53
France,16,26,22,64
Netherlands,15,7,12,34
Great Britain,14,22,29,65
Korea,13,9,10,32
Italy,12,13,15,40
Germany,12,13,8,33


In [0]:
teams_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("multiLine", "true") \
    .option("quote", '"') \
    .option("escape", '"') \
    .option("delimiter", ",") \
    .load("/mnt/parisolymics/raw-data/teams.csv")

teams_trimmed = teams_raw.select(
    col("team"),
    col("discipline"),
    col("events"),
    col("athletes"))

teams = teams_trimmed.withColumn("athletes", regexp_replace("athletes", r"[\[\]'\"']", ""))

teams.printSchema()
teams.show()

root
 |-- team: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- events: string (nullable = true)
 |-- athletes: string (nullable = true)

+--------------------+----------+------------+--------------------+
|                team|discipline|      events|            athletes|
+--------------------+----------+------------+--------------------+
|People's Republic...|   Archery|  Men's Team|KAO Wenchao, LI Z...|
|            Colombia|   Archery|  Men's Team|ARCILA Santiago, ...|
|              France|   Archery|  Men's Team|ADDIS Baptiste, C...|
|       Great Britain|   Archery|  Men's Team|HALL Conor, HALL ...|
|               India|   Archery|  Men's Team|BOMMADEVARA Dhira...|
|               Italy|   Archery|  Men's Team|MUSOLESI Federico...|
|               Japan|   Archery|  Men's Team|FURUKAWA Takaharu...|
|          Kazakhstan|   Archery|  Men's Team|ABDULLIN Ilfat, Y...|
|   Republic of Korea|   Archery|  Men's Team|KIM Je Deok, KIM ...|
|              Mexico|

In [0]:
display(teams)

team,discipline,events,athletes
People's Republic of China,Archery,Men's Team,"KAO Wenchao, LI Zhongyuan, WANG Yan"
Colombia,Archery,Men's Team,"ARCILA Santiago, ENRIQUEZ Jorge, HERNANDEZ VERA Andres"
France,Archery,Men's Team,"ADDIS Baptiste, CHIRAULT Thomas, VALLADONT Jean-Charles"
Great Britain,Archery,Men's Team,"HALL Conor, HALL Tom, WISE Alex"
India,Archery,Men's Team,"BOMMADEVARA Dhiraj, JADHAV Pravin Ramesh, RAI Tarundeep"
Italy,Archery,Men's Team,"MUSOLESI Federico, NESPOLI Mauro, PAOLI Alessandro"
Japan,Archery,Men's Team,"FURUKAWA Takaharu, NAKANISHI Junya, SAITO Fumiya"
Kazakhstan,Archery,Men's Team,"ABDULLIN Ilfat, YEREMENKO Alexandr, ZHANGBYRBAY Dauletkeldi"
Republic of Korea,Archery,Men's Team,"KIM Je Deok, KIM Woojin, LEE Wooseok"
Mexico,Archery,Men's Team,"GRANDE Matias, MARTINEZ WING Bruno, ROJAS Carlos"


In [0]:
athletes.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolymics/transformed-data/athletes.csv")

In [0]:
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolymics/transformed-data/coaches.csv")
medallists.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolymics/transformed-data/medallists.csv")
medalstotal.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolymics/transformed-data/medalstotal.csv")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolymics/transformed-data/teams.csv")